In [1]:
import gym
import ptan
import random
import numpy as np

import torch
import torch.optim as optim
import torch.nn.functional as F

from ignite.engine import Engine

from lib import common, dqn_extra

NAME = "07_distrib"

In [3]:
def calc_loss(batch, net, tgt_net, gamma, device="cpu"):
    states, actions, rewards, dones, next_states = \
        common.unpack_batch(batch)
    batch_size = len(batch)

    states_v = torch.tensor(states).to(device)
    actions_v = torch.tensor(actions).to(device)
    next_states_v = torch.tensor(next_states).to(device)

    # next state distribution
    next_distr_v, next_qvals_v = tgt_net.both(next_states_v)
    next_acts = next_qvals_v.max(1)[1].data.cpu().numpy()
    next_distr = tgt_net.apply_softmax(next_distr_v)
    next_distr = next_distr.data.cpu().numpy()

    next_best_distr = next_distr[range(batch_size), next_acts]
    dones = dones.astype(np.bool)

    proj_distr = dqn_extra.distr_projection(
        next_best_distr, rewards, dones, gamma)

    distr_v = net(states_v)
    sa_vals = distr_v[range(batch_size), actions_v.data]
    state_log_sm_v = F.log_softmax(sa_vals, dim=1)
    proj_distr_v = torch.tensor(proj_distr).to(device)

    loss_v = -state_log_sm_v * proj_distr_v
    return loss_v.sum(dim=1).mean()

In [4]:
random.seed(common.SEED)
torch.manual_seed(common.SEED)
params = common.HYPERPARAMS['pong']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

env = gym.make("PongNoFrameskip-v4")
env = ptan.common.wrappers.wrap_dqn(env)
env.seed(common.SEED)

[123, 151010689]

In [5]:
net = dqn_extra.DistributionalDQN(env.observation_space.shape, env.action_space.n).to(device)
print(net)
tgt_net = ptan.agent.TargetNet(net)
selector = ptan.actions.EpsilonGreedyActionSelector(epsilon=params.epsilon_start)
epsilon_tracker = common.EpsilonTracker(selector, params)
agent = ptan.agent.DQNAgent(lambda x: net.qvals(x), selector, device = device)

exp_source = ptan.experience.ExperienceSourceFirstLast(
    env, agent, gamma=params.gamma)
buffer = ptan.experience.ExperienceReplayBuffer(
    exp_source, buffer_size=params.replay_size)
optimizer = optim.Adam(net.parameters(), lr=params.learning_rate)

DistributionalDQN(
  (conv): Sequential(
    (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=3136, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=306, bias=True)
  )
  (softmax): Softmax(dim=1)
)


In [ ]:
def process_batch(engine, batch):
    optimizer.zero_grad()
    loss_v = calc_loss(batch, net, tgt_net.target_model,
                       gamma=params.gamma, device=device)
    loss_v.backward()
    optimizer.step()
    epsilon_tracker.frame(engine.state.iteration)
    if engine.state.iteration % params.target_net_sync == 0:
        tgt_net.sync()

    return {
        "loss": loss_v.item(),
        "epsilon": selector.epsilon,
    }

engine = Engine(process_batch)
common.setup_ignite(engine, params, exp_source, NAME)
engine.run(common.batch_generator(buffer, params.replay_initial, params.batch_size))

Episode 1: reward=-21, steps=840, speed=0.0 f/s, elapsed=0:01:12
Episode 2: reward=-21, steps=912, speed=0.0 f/s, elapsed=0:01:12
Episode 3: reward=-21, steps=955, speed=0.0 f/s, elapsed=0:01:12
Episode 4: reward=-21, steps=757, speed=0.0 f/s, elapsed=0:01:12
Episode 5: reward=-21, steps=903, speed=0.0 f/s, elapsed=0:01:12
Episode 6: reward=-20, steps=1033, speed=0.0 f/s, elapsed=0:01:12
Episode 7: reward=-20, steps=1030, speed=0.0 f/s, elapsed=0:01:12
Episode 8: reward=-20, steps=864, speed=0.0 f/s, elapsed=0:01:12
Episode 9: reward=-20, steps=987, speed=0.0 f/s, elapsed=0:01:12
Episode 10: reward=-21, steps=869, speed=0.0 f/s, elapsed=0:01:12
Episode 11: reward=-20, steps=972, speed=45.2 f/s, elapsed=0:01:15
Episode 12: reward=-21, steps=814, speed=45.3 f/s, elapsed=0:01:30
Episode 13: reward=-20, steps=1161, speed=45.5 f/s, elapsed=0:01:53
Episode 14: reward=-21, steps=984, speed=45.6 f/s, elapsed=0:02:12
Episode 15: reward=-20, steps=840, speed=45.7 f/s, elapsed=0:02:28
Episode 16: